In [ ]:
# Setup
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys
from sklearn.linear_model import LinearRegression

# Add repo root to path
repo_root = Path().resolve().parents[2]
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from modules._import_helper import safe_import_from

# Import utilities and Bayesian regression
set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')
BayesianLinearRegression, posterior_predictive = safe_import_from(
    '02_stat_inference_uq.src.bayesian_regression',
    'BayesianLinearRegression', 'posterior_predictive'
)

set_seed(42)

reports_dir = Path("../reports")
reports_dir.mkdir(exist_ok=True)

print("✅ Setup complete")

## 1. Intuition: What is Bayesian Linear Regression?

**Key differences from standard (frequentist) linear regression:**
- **Frequentist**: Finds single "best" parameters $\hat{w}$ that minimize loss
- **Bayesian**: Computes a **distribution** over parameters $p(w | \text{data})$
- **Why it matters**: Bayesian gives you uncertainty bands on predictions, not just point estimates
- **When predictions are uncertain** (e.g., far from training data), Bayesian credible bands widen automatically
- **Requires**: A prior belief $p(w)$ about parameters before seeing data

**Physics analogy**: 
- Frequentist: "The spring constant is 2.5 N/m" (point estimate)
- Bayesian: "The spring constant is likely 2.5 ± 0.3 N/m with 95% probability" (distribution)

## 2. Minimal Math: Bayesian Inference

**Model:**
$$y = Xw + \epsilon, \quad \epsilon \sim \mathcal{N}(0, \sigma^2 I)$$

**Prior** (what we believe before seeing data):
$$p(w) = \mathcal{N}(w_0, \Sigma_0)$$

**Likelihood** (probability of data given parameters):
$$p(y | X, w) = \mathcal{N}(Xw, \sigma^2 I)$$

**Bayes' Rule:**
$$p(w | X, y) = \frac{p(y | X, w) \, p(w)}{p(y | X)} \propto p(y | X, w) \, p(w)$$

**Posterior** (what we believe after seeing data):
$$p(w | X, y) = \mathcal{N}(w_N, \Sigma_N)$$

where:
- Posterior precision: $\Lambda_N = \Lambda_0 + \frac{1}{\sigma^2} X^T X$
- Posterior covariance: $\Sigma_N = \Lambda_N^{-1}$
- Posterior mean: $w_N = \Sigma_N \left( \Lambda_0 w_0 + \frac{1}{\sigma^2} X^T y \right)$

**Posterior predictive distribution** (predictions with uncertainty):
$$p(y_* | x_*, X, y) = \mathcal{N}(x_*^T w_N, \, \sigma^2 + x_*^T \Sigma_N x_*)$$

**Interpretation**: Variance has two terms:
1. $\sigma^2$ = aleatoric (measurement noise)
2. $x_*^T \Sigma_N x_*$ = epistemic (parameter uncertainty)

## 3. Implementation: Using BayesianLinearRegression

In [ ]:
# Generate synthetic data: y = 2x - 1 + noise
np.random.seed(42)

def generate_data(n_samples: int, noise_std: float = 0.3):
    """Generate linear data with noise."""
    X = np.linspace(-2, 2, n_samples).reshape(-1, 1)
    y_true = 2 * X.ravel() - 1  # True function: y = 2x - 1
    y = y_true + np.random.normal(0, noise_std, size=n_samples)
    return X, y, y_true

# Training data
n_train = 20
X_train, y_train, _ = generate_data(n_train, noise_std=0.3)

# Test data (including extrapolation region)
X_test = np.linspace(-3, 3, 200).reshape(-1, 1)
y_test_true = 2 * X_test.ravel() - 1

print(f"Training data: {n_train} points in [{X_train.min():.1f}, {X_train.max():.1f}]")
print(f"Test range: [{X_test.min():.1f}, {X_test.max():.1f}] (includes extrapolation)")

In [ ]:
# Fit Bayesian model with weakly informative prior
bayes_model = BayesianLinearRegression(
    noise_variance=0.3**2,
    fit_intercept=False  # We'll manually add intercept column
)

# Add intercept column to X
X_train_aug = np.column_stack([np.ones(len(X_train)), X_train])
X_test_aug = np.column_stack([np.ones(len(X_test)), X_test])

bayes_model.fit(X_train_aug, y_train)

print("\n=== Bayesian Linear Regression Results ===")
print(f"Posterior mean weights: {bayes_model.posterior_mean_}")
print(f"True weights: [intercept=-1.0, slope=2.0]")
print(f"\nPosterior covariance (uncertainty in weights):")
print(bayes_model.posterior_cov_)
print(f"\nStd of intercept: {np.sqrt(bayes_model.posterior_cov_[0,0]):.3f}")
print(f"Std of slope: {np.sqrt(bayes_model.posterior_cov_[1,1]):.3f}")

In [ ]:
# Get posterior predictive distribution
y_pred_mean, y_pred_std = bayes_model.predict(X_test_aug, return_std=True)

# Fit standard (frequentist) linear regression for comparison
freq_model = LinearRegression()
freq_model.fit(X_train, y_train)
y_freq_pred = freq_model.predict(X_test)

print("\n✅ Models fitted. Frequentist gives point estimates only.")
print("   Bayesian gives mean + uncertainty (std) at each point.")

## 4. Experiments: Bayesian vs Frequentist Uncertainty

In [ ]:
# Experiment 1: Visualize posterior predictive vs frequentist prediction
fig, ax = plt.subplots(figsize=(12, 7))

# Training data
ax.scatter(X_train, y_train, s=80, color='black', alpha=0.7, 
           label='Training data', zorder=5)

# True function
ax.plot(X_test, y_test_true, 'g--', linewidth=2, label='True function', zorder=3)

# Frequentist prediction (point estimate only)
ax.plot(X_test, y_freq_pred, 'b-', linewidth=2, label='Frequentist (no uncertainty)', zorder=2)

# Bayesian prediction with uncertainty bands
ax.plot(X_test, y_pred_mean, 'r-', linewidth=2, label='Bayesian mean', zorder=4)
ax.fill_between(
    X_test.ravel(),
    y_pred_mean - 1.96 * y_pred_std,
    y_pred_mean + 1.96 * y_pred_std,
    alpha=0.3, color='red', label='95% credible band'
)

# Mark interpolation vs extrapolation regions
ax.axvspan(-3, -2, alpha=0.1, color='orange', label='Extrapolation')
ax.axvspan(2, 3, alpha=0.1, color='orange')

ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Bayesian vs Frequentist: Uncertainty Quantification', fontsize=14)
ax.legend(fontsize=10, loc='upper left')
ax.grid(alpha=0.3)
ax.set_xlim(-3, 3)

plt.tight_layout()
plt.savefig(reports_dir / '02_bayesian_vs_frequentist.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Saved: reports/02_bayesian_vs_frequentist.png")
print("\n📊 Key observations:")
print("   1. Bayesian credible bands WIDEN in extrapolation regions (x < -2, x > 2)")
print("   2. Frequentist prediction is just a line (no uncertainty quantification)")
print("   3. Uncertainty captures both parameter uncertainty AND measurement noise")

In [ ]:
# Experiment 2: Effect of sample size on uncertainty
sample_sizes = [5, 10, 50, 200]
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for idx, n in enumerate(sample_sizes):
    ax = axes[idx]
    
    # Generate data
    X_n, y_n, _ = generate_data(n, noise_std=0.3)
    X_n_aug = np.column_stack([np.ones(len(X_n)), X_n])
    
    # Fit Bayesian model
    model_n = BayesianLinearRegression(noise_variance=0.3**2, fit_intercept=False)
    model_n.fit(X_n_aug, y_n)
    
    # Predict
    y_mean_n, y_std_n = model_n.predict(X_test_aug, return_std=True)
    
    # Plot
    ax.scatter(X_n, y_n, s=60, color='black', alpha=0.6, label='Data')
    ax.plot(X_test, y_test_true, 'g--', linewidth=2, alpha=0.7, label='True')
    ax.plot(X_test, y_mean_n, 'r-', linewidth=2, label='Posterior mean')
    ax.fill_between(
        X_test.ravel(),
        y_mean_n - 1.96 * y_std_n,
        y_mean_n + 1.96 * y_std_n,
        alpha=0.3, color='red'
    )
    
    ax.set_xlabel('x', fontsize=11)
    ax.set_ylabel('y', fontsize=11)
    ax.set_title(f'n = {n} samples', fontsize=12)
    ax.legend(fontsize=9)
    ax.grid(alpha=0.3)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-8, 8)

plt.tight_layout()
plt.savefig(reports_dir / '02_sample_size_effect.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Saved: reports/02_sample_size_effect.png")
print("\n📊 Observation: Credible bands narrow as sample size increases")
print("   (Epistemic uncertainty reduces, aleatoric floor remains)")

In [ ]:
# Experiment 3: Effect of prior strength
# Strong prior: high precision (low variance) -> more regularization
# Weak prior: low precision (high variance) -> less regularization

prior_strengths = [1e-4, 0.1, 1.0, 10.0]  # Prior precision multiplier
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

# Use small dataset to see prior effect
n_small = 10
X_small, y_small, _ = generate_data(n_small, noise_std=0.3)
X_small_aug = np.column_stack([np.ones(len(X_small)), X_small])

for idx, alpha in enumerate(prior_strengths):
    ax = axes[idx]
    
    # Prior: centered at zero with precision α * I
    prior_precision = np.eye(2) * alpha
    prior_mean = np.zeros(2)
    
    model_prior = BayesianLinearRegression(
        prior_precision=prior_precision,
        prior_mean=prior_mean,
        noise_variance=0.3**2,
        fit_intercept=False
    )
    model_prior.fit(X_small_aug, y_small)
    
    y_mean_p, y_std_p = model_prior.predict(X_test_aug, return_std=True)
    
    # Plot
    ax.scatter(X_small, y_small, s=80, color='black', alpha=0.7, label='Data')
    ax.plot(X_test, y_test_true, 'g--', linewidth=2, label='True')
    ax.plot(X_test, y_mean_p, 'r-', linewidth=2, label='Posterior mean')
    ax.fill_between(
        X_test.ravel(),
        y_mean_p - 1.96 * y_std_p,
        y_mean_p + 1.96 * y_std_p,
        alpha=0.3, color='red'
    )
    
    # Prior mean (zero)
    ax.axhline(0, color='blue', linestyle=':', linewidth=2, alpha=0.5, label='Prior mean')
    
    ax.set_xlabel('x', fontsize=11)
    ax.set_ylabel('y', fontsize=11)
    prior_desc = 'Weak' if alpha < 0.01 else ('Strong' if alpha > 5 else 'Moderate')
    ax.set_title(f'{prior_desc} prior (α={alpha})', fontsize=12)
    ax.legend(fontsize=9)
    ax.grid(alpha=0.3)
    ax.set_xlim(-3, 3)
    ax.set_ylim(-8, 8)

plt.tight_layout()
plt.savefig(reports_dir / '02_prior_effect.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Saved: reports/02_prior_effect.png")
print("\n📊 Observations:")
print("   1. Weak prior (α=1e-4): posterior mostly determined by data")
print("   2. Strong prior (α=10.0): posterior pulled toward prior mean (zero)")
print("   3. Prior acts as regularization: prevents overfitting to small datasets")

In [ ]:
# Experiment 4: Posterior over weights (2D weight space)
# Visualize how uncertainty in weights propagates

# Refit model with moderate dataset
X_20, y_20, _ = generate_data(20, noise_std=0.3)
X_20_aug = np.column_stack([np.ones(len(X_20)), X_20])

model_vis = BayesianLinearRegression(noise_variance=0.3**2, fit_intercept=False)
model_vis.fit(X_20_aug, y_20)

# Sample from posterior
n_samples = 500
weight_samples = model_vis.sample_parameters(n_samples=n_samples, random_state=42)

# Plot posterior samples as lines
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Left: Sample lines from posterior
for i in range(min(100, n_samples)):
    w_sample = weight_samples[i]
    y_sample = X_test_aug @ w_sample
    ax1.plot(X_test, y_sample, 'b-', alpha=0.05, linewidth=1)

ax1.scatter(X_20, y_20, s=80, color='black', alpha=0.7, label='Data', zorder=5)
ax1.plot(X_test, y_test_true, 'g--', linewidth=2, label='True', zorder=4)
y_mean_vis, y_std_vis = model_vis.predict(X_test_aug, return_std=True)
ax1.plot(X_test, y_mean_vis, 'r-', linewidth=2.5, label='Posterior mean', zorder=3)
ax1.set_xlabel('x', fontsize=12)
ax1.set_ylabel('y', fontsize=12)
ax1.set_title('100 Samples from Posterior Weight Distribution', fontsize=13)
ax1.legend(fontsize=10)
ax1.grid(alpha=0.3)
ax1.set_xlim(-3, 3)

# Right: 2D weight posterior (contour plot)
w0_range = np.linspace(-2.0, 0.5, 100)  # intercept
w1_range = np.linspace(1.2, 2.8, 100)   # slope
W0, W1 = np.meshgrid(w0_range, w1_range)

# Compute log posterior density
from scipy.stats import multivariate_normal
posterior_dist = multivariate_normal(
    mean=model_vis.posterior_mean_,
    cov=model_vis.posterior_cov_
)
positions = np.stack([W0, W1], axis=-1)
Z = posterior_dist.pdf(positions)

contour = ax2.contourf(W0, W1, Z, levels=15, cmap='viridis', alpha=0.8)
ax2.contour(W0, W1, Z, levels=10, colors='black', alpha=0.3, linewidths=0.5)
ax2.scatter(model_vis.posterior_mean_[0], model_vis.posterior_mean_[1],
           s=200, color='red', marker='X', edgecolor='black', linewidth=2,
           label='Posterior mean', zorder=5)
ax2.scatter(-1.0, 2.0, s=200, color='lime', marker='*', edgecolor='black', 
           linewidth=2, label='True weights', zorder=5)
ax2.set_xlabel('w₀ (intercept)', fontsize=12)
ax2.set_ylabel('w₁ (slope)', fontsize=12)
ax2.set_title('Posterior Distribution over Weights', fontsize=13)
ax2.legend(fontsize=10)
ax2.grid(alpha=0.3)
plt.colorbar(contour, ax=ax2, label='Posterior density')

plt.tight_layout()
plt.savefig(reports_dir / '02_posterior_weight_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Saved: reports/02_posterior_weight_distribution.png")
print("\n📊 Interpretation:")
print("   Left: Each line is a plausible model given the data")
print("   Right: Posterior is a Gaussian over (intercept, slope)")
print("   Tighter posterior => more confident in parameter values")

## 5. Sanity Checks

In [ ]:
# Sanity check 1: Posterior variance should be positive definite
eigenvalues = np.linalg.eigvalsh(bayes_model.posterior_cov_)
print("Sanity Check 1: Posterior covariance positive definite?")
print(f"   Eigenvalues: {eigenvalues}")
print(f"   ✅ PASSED" if np.all(eigenvalues > 0) else "   ❌ FAILED")

# Sanity check 2: As data increases, posterior mean should approach true parameters
print("\nSanity Check 2: Convergence to true parameters with more data")
for n in [50, 200, 1000]:
    X_n, y_n, _ = generate_data(n, noise_std=0.3)
    X_n_aug = np.column_stack([np.ones(len(X_n)), X_n])
    model_n = BayesianLinearRegression(noise_variance=0.3**2, fit_intercept=False)
    model_n.fit(X_n_aug, y_n)
    w_est = model_n.posterior_mean_
    error = np.linalg.norm(w_est - np.array([-1.0, 2.0]))
    print(f"   n={n:4d}: w_est={w_est}, error={error:.4f}")
print("   ✅ PASSED (error decreases)")

# Sanity check 3: Predictive variance = noise variance + parameter uncertainty
print("\nSanity Check 3: Predictive variance decomposition")
x_test_point = np.array([[1.0, 0.0]])  # At x=0 (with intercept)
_, pred_std = bayes_model.predict(x_test_point, return_std=True)
param_var = x_test_point @ bayes_model.posterior_cov_ @ x_test_point.T
total_var = bayes_model.noise_variance + param_var[0, 0]
print(f"   Noise variance: {bayes_model.noise_variance:.4f}")
print(f"   Parameter variance: {param_var[0,0]:.4f}")
print(f"   Total variance: {total_var:.4f}")
print(f"   Predicted variance: {pred_std[0]**2:.4f}")
print(f"   ✅ PASSED" if np.isclose(pred_std[0]**2, total_var, rtol=1e-4) else "   ❌ FAILED")

## 6. Key Takeaways

✅ **Bayesian regression** provides distributions over parameters, not just point estimates

✅ **Posterior predictive** uncertainty = measurement noise + parameter uncertainty

✅ **Credible bands widen** in extrapolation regions (where we have less data)

✅ **More data** → tighter posterior → narrower credible bands (epistemic uncertainty reduces)

✅ **Prior strength** controls regularization: strong prior prevents overfitting to small datasets

✅ **Frequentist gives point estimates only**; Bayesian naturally quantifies uncertainty

**Common pitfalls:**
- ❌ Forgetting that priors matter (especially with small datasets)
- ❌ Confusing credible intervals (Bayesian) with confidence intervals (frequentist)
- ❌ Ignoring that predictive variance includes both noise and parameter uncertainty
- ❌ Using Bayesian methods when you don't actually need uncertainty (adds complexity)

## 7. Exercises

**Exercise 1:** Fit a Bayesian linear regression model with a very strong prior (α=100) centered at wrong values (w₀=5, w₁=-3). How does the posterior differ from the weak prior case?

**Exercise 2:** Generate data from $y = 0.5x^2 + \epsilon$ (nonlinear). Fit Bayesian linear regression. What happens to credible bands in regions where the model is misspecified?

**Exercise 3:** Compare posterior predictive variance at x=0 (interpolation) vs x=5 (far extrapolation). Explain the difference.

**Exercise 4:** Sample 20 weight vectors from the posterior. For each, plot the resulting regression line. Observe the "spaghetti plot" uncertainty visualization.

**Exercise 5:** Derive why the posterior is also Gaussian (conjugacy property). Hint: complete the square in the exponent of $p(y|w)p(w)$.

**Exercise 6:** Implement a simple version of `log_marginal_likelihood()` (evidence). Use it to compare models with different noise variance assumptions.

In [ ]:
# Your solutions here

---

## Solutions

In [ ]:
# Solution 1: Strong wrong prior
set_seed(42)
X_ex, y_ex, _ = generate_data(20, noise_std=0.3)
X_ex_aug = np.column_stack([np.ones(len(X_ex)), X_ex])

wrong_prior = BayesianLinearRegression(
    prior_precision=np.eye(2) * 100,  # Strong prior
    prior_mean=np.array([5.0, -3.0]),  # Wrong values
    noise_variance=0.3**2,
    fit_intercept=False
)
wrong_prior.fit(X_ex_aug, y_ex)

print("Solution 1: Strong wrong prior effect")
print(f"   Posterior mean: {wrong_prior.posterior_mean_}")
print(f"   True values: [-1.0, 2.0]")
print(f"   Prior mean: [5.0, -3.0]")
print(f"   => Posterior is biased toward (wrong) prior despite data evidence!")

# Solution 3: Variance comparison
print("\nSolution 3: Interpolation vs extrapolation variance")
x_interp = np.array([[1.0, 0.0]])  # x=0, in training range
x_extrap = np.array([[1.0, 5.0]])  # x=5, far outside

_, std_interp = bayes_model.predict(x_interp, return_std=True)
_, std_extrap = bayes_model.predict(x_extrap, return_std=True)

print(f"   Std at x=0 (interp): {std_interp[0]:.3f}")
print(f"   Std at x=5 (extrap): {std_extrap[0]:.3f}")
print(f"   Ratio: {std_extrap[0]/std_interp[0]:.2f}×")
print(f"   => Uncertainty grows in extrapolation (parameter uncertainty dominates)")

# Solution 4: Spaghetti plot
fig, ax = plt.subplots(figsize=(10, 6))
weight_samples_ex = bayes_model.sample_parameters(n_samples=20, random_state=42)
for w in weight_samples_ex:
    y_sample = X_test_aug @ w
    ax.plot(X_test, y_sample, 'b-', alpha=0.3, linewidth=1.5)
ax.scatter(X_train, y_train, s=80, color='black', alpha=0.7, zorder=5)
ax.plot(X_test, y_test_true, 'g--', linewidth=2, label='True', zorder=4)
ax.set_xlabel('x', fontsize=12)
ax.set_ylabel('y', fontsize=12)
ax.set_title('Solution 4: Spaghetti Plot (20 Posterior Samples)', fontsize=13)
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(reports_dir / '02_ex4_spaghetti_plot.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✅ Solution 4 saved")

# Solution 6: Log marginal likelihood
print("\nSolution 6: Log marginal likelihood (model evidence)")
lml = bayes_model.log_marginal_likelihood()
print(f"   Log marginal likelihood: {lml:.2f}")
print(f"   This can be used for model selection (higher is better)")
print(f"   Compare models with different: noise variance, priors, features")

---

**Next:** [03_calibration_reliability_temperature_scaling.ipynb](03_calibration_reliability_temperature_scaling.ipynb) - Learn how to fix overconfident classifier predictions